In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Player 클래스: 환경 상태 관리
# -------------------------
class Player(object):
    def __init__(self, currentSum, usableAce, dealersCard):
        # 현재 플레이어 점수
        self.currentSum = currentSum
        # 딜러 오픈 카드
        self.dealersCard = dealersCard
        # 에이스 사용 가능 여부
        self.usableAce = usableAce
        # 실제 게임 중 에이스 사용 여부
        self.usingAce = self.usableAce

    def ReceiveCard(self, card):
        # 카드를 받으면 점수 갱신
        if self.usingAce and self.currentSum + card > 21:
            # 에이스를 1로 처리하여 버스트 방지
            self.usingAce = False
            self.currentSum += card - 10
        else:
            self.currentSum += card

    def GetState(self):
        # MC 학습에 사용될 상태 표현 (currentSum, usableAce, dealersCard)
        return (self.currentSum, self.usableAce, self.dealersCard)

    def GetValue(self):
        return self.currentSum

    def ShouldHit(self, policy):
        # 현재 상태에서 정책에 따라 히트 여부 결정
        return policy[self.GetState()]

    def Bust(self):
        # 플레이어 버스트 여부 확인
        return self.GetValue() > 21


# -------------------------
# Dealer 클래스: 환경 샘플링용
# -------------------------
class Dealer(object):
    def __init__(self, cards):
        self.cards = cards

    def ReceiveCard(self, card):
        # 딜러 점수 업데이트
        self.cards.append(card)

    def GetValue(self):
        # 딜러 점수 계산, 에이스 처리
        currentSum = 0
        aceCount = 0

        for card in self.cards:
            if card == 1:
                aceCount += 1
            else:
                currentSum += card

        while aceCount > 0:
            aceCount -= 1
            currentSum += 11

            if currentSum > 21:
                aceCount += 1
                currentSum -= 11
                currentSum += aceCount
                break

        return currentSum

    def ShouldHit(self):
        # 딜러 정책: 17 이상이면 스탑
        if self.GetValue() >= 17:
            return False
        else:
            return True

    def Bust(self):
        return self.GetValue() > 21


# -------------------------
# 상태-행동 쌍 기록용 클래스
# -------------------------
class StateActionInfo(object):
    def __init__(self):
        # 상태-행동 쌍 리스트
        self.stateActionPairs = []
        # 중복 확인용 집합
        self.stateActionMap = set()

    def AddPair(self, pair):
        # 중복 추가 방지
        if pair in self.stateActionMap:
            return

        self.stateActionPairs.append(pair)
        self.stateActionMap.add(pair)


# -------------------------
# MC 학습 핵심: 에피소드 종료 후 Q값 업데이트
# -------------------------
def EvaluateAndImprovePolicy(qMap, policy, returns, stateActionPairs, reward):
    for pair in stateActionPairs:
        # 상태-행동 쌍 방문 횟수 증가
        returns[pair] += 1
        # MC 학습: Q(s,a) 점진적 업데이트
        qMap[pair] = qMap[pair] + ((reward - qMap[pair]) / returns[pair])

        state = pair[0]
        shouldHit = False

        # 현재 상태에서 Q값 비교 → 정책 갱신
        if qMap[(state, True)] > qMap[(state, False)]: # true -> 히트일 때 보상, false -> 스탠드일 때 보상
            shouldHit = True

        policy[state] = shouldHit


# -------------------------
# 환경 카드 샘플링
# -------------------------
def newCard():
    card = np.random.randint(1, 14)
    # 10, J, Q, K 모두 10점 처리
    if card > 9:
        return 10
    else:
        return card


# -------------------------
# MC 학습을 위한 한 에피소드 진행
# -------------------------
def PlayEpisode(qMap, policy, returns):
    # 초기 상태 샘플링
    playerSum = np.random.randint(11, 22)
    dealerOpenCard = np.random.randint(1, 11)
    usableAce = bool(np.random.randint(0, 2))

    player = Player(playerSum, usableAce, dealerOpenCard)
    dealer = Dealer([dealerOpenCard])

    # 상태-행동 쌍 기록
    stateActionInfo = StateActionInfo()
    hitAction = bool(np.random.randint(0, 2))  # 초기 행동 랜덤 선택
    stateActionInfo.AddPair((player.GetState(), hitAction))

    if hitAction:
        player.ReceiveCard(newCard())

        # 정책에 따라 계속 히트
        while not player.Bust() and player.ShouldHit(policy):
            stateActionInfo.AddPair((player.GetState(), True))
            player.ReceiveCard(newCard())

    # 플레이어 버스트 시 MC 업데이트
    if player.Bust():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, -1)
        return

    # 히트 안함 상태 기록
    stateActionInfo.AddPair((player.GetState(), False))
    dealer.ReceiveCard(newCard())

    # 딜러 정책 시뮬레이션
    while not dealer.Bust() and dealer.ShouldHit():
        dealer.cards.append(newCard())

    # 에피소드 종료 후 보상 계산 및 MC 업데이트
    if dealer.Bust() or dealer.GetValue() < player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, 1)
    elif dealer.GetValue() > player.GetValue():
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, -1)
    else:
        EvaluateAndImprovePolicy(qMap, policy, returns, stateActionInfo.stateActionPairs, 0)


# -------------------------
# Q값, 정책, 방문 횟수 초기화
# -------------------------
qMap = {}
policy = {}
returns = {}

for playerSum in range(11, 22):
    for usableAce in range(2):
        for dealersCard in range(1, 11):
            playerState = (playerSum, bool(usableAce), dealersCard)
            qMap[(playerState, False)] = 0
            qMap[(playerState, True)] = 0
            returns[(playerState, False)] = 0
            returns[(playerState, True)] = 0

            # 초기 정책: 20,21이면 스탑, 아니면 히트
            if playerSum == 20 or playerSum == 21:
                policy[playerState] = False
            else:
                policy[playerState] = True

# -------------------------
# MC 학습 반복
# -------------------------
for i in range(100000):
    PlayEpisode(qMap, policy, returns)

# -------------------------
# 시각화 준비
# -------------------------
x11 = []
y11 = []
x12 = []
y12 = []
x21 = []
y21 = []
x22 = []
y22 = []

for playerState in policy:
    if playerState[1]:
        if policy[playerState]:
            x11.append(playerState[2] - 1)
            y11.append(playerState[0] - 11)
        else:
            x12.append(playerState[2] - 1)
            y12.append(playerState[0] - 11)
    else:
        if policy[playerState]:
            x21.append(playerState[2] - 1)
            y21.append(playerState[0] - 11)
        else:
            x22.append(playerState[2] - 1)
            y22.append(playerState[0] - 11)

# -------------------------
# 결과 시각화
# -------------------------
plt.figure(0)
plt.title('With Usable Ace')
plt.scatter(x11, y11, color='blue')
plt.scatter(x12, y12, color='yellow')
plt.xticks(range(10), ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
plt.yticks(range(11), [str(i) for i in range(11, 22)])

plt.figure(1)
plt.title('Without Usable Ace')
plt.scatter(x21, y21, color='blue')
plt.scatter(x22, y22, color='yellow')
plt.xticks(range(10), ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
plt.yticks(range(11), [str(i) for i in range(11, 22)])

plt.show()
